<a href="https://colab.research.google.com/github/MarioSolano98/actividad-evaluativa3-AC/blob/main/actividad_evaluativa3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Punto 4

## Importar Librerias

In [ ]:
# Clonar el repositorio
!git clone https://github.com/MarioSolano98/actividad-evaluativa3-AC.git

In [ ]:
# Importar librerías esenciales
import pandas as pd            # Para manipulación y análisis de datos
import numpy as np             # Para operaciones numéricas y manejo de arrays
import matplotlib.pyplot as plt # Para visualización de gráficos
import seaborn as sns          # Para gráficos estadísticos y de exploración de datos
from sklearn.model_selection import train_test_split # Para dividir datos en entrenamiento y prueba
from sklearn.linear_model import LogisticRegression  # Modelo Logit básico
from sklearn.linear_model import RidgeClassifier, Lasso, ElasticNet # Modelos logísticos penalizados
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score # Para evaluar modelos
from sklearn.preprocessing import StandardScaler    # Para normalizar datos
from sklearn.model_selection import GridSearchCV    # Para optimizar hiperparámetros

## Análisis exploratorio

In [ ]:
# Cargar el archivo de datos
data_path = '/content/actividad-evaluativa3-AC/Dengue_Data.xlsx'
data = pd.read_excel(data_path)

# Visualizar las primeras filas
data.head()

In [ ]:
# Información general del DataFrame
print("Información general del dataset:")
data.info()


In [ ]:
# Estadísticas descriptivas de las variables numéricas
print("\nEstadísticas descriptivas:")
data.describe()


In [ ]:
# Visualización de histogramas para variables numéricas
data.hist(bins=20, figsize=(15, 10))
plt.show()

In [ ]:
# Visualización de histogramas para variables categoricas

# Selección de las columnas categóricas
categorical_columns = data.select_dtypes(include=['object']).columns

# Configuración de subplots en grupos de tres
n_cols = 3
n_rows = (len(categorical_columns) + n_cols - 1) // n_cols
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(18, 6 * n_rows))

# Aplanar los ejes en una sola lista para iterar fácilmente
axes = axes.flatten()

# Crear gráficos de barras para cada variable categórica
for i, col in enumerate(categorical_columns):
    sns.countplot(data=data, x=col, ax=axes[i])
    axes[i].set_title(f'Frecuencia de {col}')
    axes[i].tick_params(axis='x', rotation=45)
    # Ajuste del límite superior de ylim para que se vea más espaciado
    max_count = data[col].value_counts().max()
    axes[i].set_ylim(0, max_count * 1.2)  # Ajuste del límite superior al 120% del valor máximo

# Ocultar cualquier gráfico adicional en el subplot si hay un número impar de gráficos
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

# Ajuste del espaciado para evitar superposiciones
plt.tight_layout(h_pad=5)  # h_pad controla el espacio vertical entre gráficos
plt.show()

#Identificación de valores nulos.
Seleccionamos las columnas en las cuales los valores nulos representen mas del 50% del total de los datos

In [ ]:
#Identificación de las columnas con datos Nulos

# Calcular el umbral del 50% en función del número de filas
threshold = len(data) * 0.5

# Contar los valores nulos en cada columna y filtrar aquellas que superen el umbral del 50%
null_counts = data.isnull().sum()
high_null_columns = null_counts[null_counts > threshold].index.tolist()

# Mostrar las columnas que tienen más del 50% de valores nulos
print("Columnas con más del 50% de valores nulos:", high_null_columns)

# Gráfico de barras solo para columnas con valores nulos
plt.figure(figsize=(30, 6))
sns.barplot(x=null_counts.index, y=null_counts.values, palette="viridis")
plt.xticks(rotation=45)
plt.xlabel("Variables con valores nulos")
plt.ylabel("Frecuencia de valores nulos")
plt.title("Frecuencia de valores nulos en variables seleccionadas")
plt.show()

In [ ]:
# Eliminamos las columnas en las cuales los datos nulos sean mayores al 50% de sus datos.
data_filtered = data.drop(columns=high_null_columns)
data_filtered


#Eliminamos columnas repetidas

In [ ]:
#Eliminamos las columnas repetidas en este caso las de "área_"

# Eliminar las columnas 'area_' y 'area_.1'
data_filtered = data_filtered.drop(columns=['area_', 'area_.1'], errors='ignore')

# Renombrar la columna 'area_.2' a 'area_'
data_filtered = data_filtered.rename(columns={'area_.2': 'area_'})

data_filtered


In [ ]:
# Identificar columnas que contienen "No" o "Sí"
filtered_columns = [col for col in data_filtered.columns if data_filtered[col].astype(str).str.contains("No|Sí", case=False, na=False).any()]

# Cambiar "Sí" a 1 y "No" o nulos a 0 en las columnas identificadas
for col in filtered_columns:
    data_filtered[col] = data_filtered[col].apply(lambda x: 1 if str(x).strip().lower() == 'sí' else 0)

# Mostrar las primeras filas para verificar el resultado
data_filtered[filtered_columns].head()
data_filtered

In [ ]:
#Convertimos las demas variables categoricas en numericas (Grupos edad, sexo_, area_, y nom_eve)
from sklearn.preprocessing import LabelEncoder

# Identificamos columnas categóricas que necesitan codificación
categorical_columns = data_filtered.select_dtypes(include=['object']).columns

# Aplicamos codificación de etiquetas a cada columna categórica
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data_filtered[col] = le.fit_transform(data_filtered[col])
    label_encoders[col] = le  # Guardamos el codificador para hacer referencia al mapeo más adelante si es necesario

# Muestra el marco de datos transformado y las asignaciones para cada columna codificada.
data_filtered.head(), {col: list(le.classes_) for col, le in label_encoders.items()}

Valores asignados a cada categoria:

Grupos edad:

*   Adolescencia: 0
*   Adultez: 1
*   Infancia: 2
*   Juventud: 3
*   Primera infancia: 4
*   Vejez: 5


sexo_:

* F (Femenino): 0
* M (Masculino): 1


area_:

* Cabecera Municipal: 0
* Centro Poblado: 1
* Rural Disperso: 2

nom_eve:

* DENGUE: 0
* DENGUE GRAVE: 1

In [ ]:
data_filtered.head()

In [ ]:
data_filtered.info()

## Identificación de variables "Sociodemograficas" y "Clinicas"

In [ ]:
# Identificar variables socio-demográficas y clínicas

# Listado de las columnas
columnas = data_filtered.columns

# Variables socio-demográficas (basadas en nombres típicos y el contenido inspeccionado)
variables_socio_demograficas = [
    'edad_', 'Grupos edad', 'sexo_', 'area_', 'ocupacion_', 'per_etn_',
    'gp_discapa', 'gp_desplaz', 'gp_migrant', 'gp_carcela', 'gp_indigen',
    'gp_pobicbf', 'gp_desmovi', 'gp_vic_vio', 'gp_otros', 'Region'
]

# Variables clínicas/médicas (basadas en nombres de síntomas y el contenido inspeccionado)
variables_clinicas = [
    'fiebre', 'cefalea', 'dolrretroo', 'malgias', 'artralgia', 'erupcionr',
    'dolor_abdo', 'vomito', 'diarrea', 'somnolenci', 'hipotensio',
    'hepatomeg', 'hem_mucosa', 'hipotermia', 'caida_plaq', 'acum_liqui',
    'aum_hemato'
]

# Variable objetivo para el primer análisis (Dengue o Dengue Grave)
variable_objetivo_dengue = 'nom_eve'

# Variable objetivo para el segundo análisis (Hospitalización)
variable_objetivo_hospitalizacion = 'pac_hos_'

variables_socio_demograficas, variables_clinicas


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


variable_objetivo_dengue = 'nom_eve'

# Filtrar los datos para las variables relevantes
X = data_filtered[variables_socio_demograficas]
y = data_filtered[variable_objetivo_dengue]

# Separar los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Definir modelos con regularización
models = {
    'Logit': LogisticRegression(penalty=None, random_state=42), # Changed 'none' to None
    'Logit Ridge': LogisticRegression(penalty='l2', C=1.0, random_state=42),
    'Logit LASSO': LogisticRegression(penalty='l1', solver='saga', C=1.0, random_state=42),
    'Logit Elastic Net': LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, C=1.0, random_state=42)
}

# Ajustar cada modelo y obtener los resultados
model_results = {}
for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    report = classification_report(y_test, y_pred, output_dict=True)
    model_results[model_name] = {
        'model': model,
        'classification_report': report,
        'coefficients': model.coef_
    }

model_results


# Crear un DataFrame para almacenar los resultados de cada modelo
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision_0', 'Recall_0', 'F1_0', 'Precision_1', 'Recall_1', 'F1_1'])

# Extraer métricas clave del reporte de clasificación de cada modelo
for model_name, results in model_results.items():
    report = results['classification_report']
    # Obtener las métricas de precisión, recall y F1 para cada clase (0 y 1)
    accuracy = report['accuracy']
    precision_0 = report['0']['precision']
    recall_0 = report['0']['recall']
    f1_0 = report['0']['f1-score']
    precision_1 = report['1']['precision']
    recall_1 = report['1']['recall']
    f1_1 = report['1']['f1-score']

    # Agregar los resultados al DataFrame using pd.concat
    results_df = pd.concat([results_df, pd.DataFrame([{
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision_0': precision_0,
        'Recall_0': recall_0,
        'F1_0': f1_0,
        'Precision_1': precision_1,
        'Recall_1': recall_1,
        'F1_1': f1_1
    }])], ignore_index=True)

results_df

# Conclusiones y Análisis:
* Alta precisión general (0.99418): La precisión en todos los modelos es alta debido a la correcta clasificación de la clase mayoritaria (0 - "DENGUE"), mientras que ningún modelo logró clasificar correctamente la clase minoritaria (1 - "DENGUE GRAVE").

* Desequilibrio de clases: La clase 1 no fue predicha correctamente por ningún modelo, reflejando el desbalance en los datos y la dificultad de los modelos para captar patrones en la clase minoritaria sin ajustes adicionales.

* Regularización: Aunque se aplicaron penalizaciones (Ridge, LASSO y Elastic Net) para mejorar la estabilidad y selección de variables, no lograron mejorar la detección de la clase minoritaria.

In [ ]:
# Definir un rango de valores de regularización para analizar la evolución de coeficientes
alphas = np.logspace(-4, 1, 100)

# Inicializar estructuras para almacenar coeficientes en función de la penalización para cada modelo
coef_ridge, coef_lasso, coef_enet = [], [], []

# Calcular y almacenar coeficientes para Ridge, LASSO y Elastic Net
for alpha in alphas:
    # Ridge
    ridge_model = LogisticRegression(penalty='l2', C=1/alpha, random_state=42, max_iter=10000)
    ridge_model.fit(X_train_scaled, y_train)
    coef_ridge.append(ridge_model.coef_.flatten())

    # LASSO
    lasso_model = LogisticRegression(penalty='l1', solver='saga', C=1/alpha, random_state=42, max_iter=10000)
    lasso_model.fit(X_train_scaled, y_train)
    coef_lasso.append(lasso_model.coef_.flatten())

    # Elastic Net
    enet_model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, C=1/alpha, random_state=42, max_iter=10000)
    enet_model.fit(X_train_scaled, y_train)
    coef_enet.append(enet_model.coef_.flatten())

# Convertir listas de coeficientes en arrays para facilitar el trazado
coef_ridge = np.array(coef_ridge)
coef_lasso = np.array(coef_lasso)
coef_enet = np.array(coef_enet)

# Nombres de las variables para etiquetas
variable_names = X.columns

# Graficar evolución de coeficientes para cada modelo
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
fig.suptitle("Evolución de Coeficientes en Función de la Penalización (α)")

# Gráfico para Ridge
axes[0].set_title("Ridge")
for i, coef in enumerate(coef_ridge.T):
    axes[0].plot(alphas, coef, label=variable_names[i])
axes[0].set_xscale("log")
axes[0].invert_xaxis()
axes[0].set_xlabel("Penalización (α)")
axes[0].set_ylabel("Coeficiente")

# Gráfico para LASSO
axes[1].set_title("LASSO")
for i, coef in enumerate(coef_lasso.T):
    axes[1].plot(alphas, coef, label=variable_names[i])
axes[1].set_xscale("log")
axes[1].invert_xaxis()
axes[1].set_xlabel("Penalización (α)")

# Gráfico para Elastic Net
axes[2].set_title("Elastic Net")
for i, coef in enumerate(coef_enet.T):
    axes[2].plot(alphas, coef, label=variable_names[i])
axes[2].set_xscale("log")
axes[2].invert_xaxis()
axes[2].set_xlabel("Penalización (α)")

# Agregar leyendas
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1), ncol=1)

plt.tight_layout()
plt.show()